In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from maps import map_robinson
from paths import file_ex_atm_ctrl, path_results
from timeseries import IterateOutputCESM
from xr_regression import xr_linear_trends_2D, xr_linear_trend

In [ ]:
# for run in ['ctrl', 'rcp']:
#     first_year = IterateOutputCESM(domain='atm', run=run, tavg='yrly').year
#     for y, m, file in IterateOutputCESM(domain='atm', run=run, tavg='yrly'):
#         ds = xr.open_dataset(file, decode_times=False)
#         ds_new = xr.merge([ds.U[-1,:,:], ds.V[-1,:,:]])
#         if y==first_year:  ds_out = ds_new
#         else:              ds_out = xr.concat([ds_out, ds_new], dim='time')
#     ds_out.to_netcdf(path=f'{path_results}/winds/surface_winds_{run}.nc', mode='w')

In [ ]:
winds_ctrl = xr.open_dataset(f'{path_results}/winds/surface_winds_ctrl.nc', decode_times=False)
winds_rcp  = xr.open_dataset(f'{path_results}/winds/surface_winds_rcp.nc' , decode_times=False)

In [ ]:
winds_rcp

In [ ]:
label = 'avg. zonal wind velocity [m/s]'
minv, maxv = -15,15
cmap = 'BrBG'
filename = f'{path_results}/winds/zonal_wind_map_ctrl'
field = winds_ctrl.U[:,:,:].mean(dim='time')

fig = map_robinson(xr_DataArray=field, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

In [ ]:
label = 'avg. meridional wind velocity [m/s]'
minv, maxv = -10,10
cmap = 'PRGn'
filename = f'{path_results}/winds/meridional_wind_map_ctrl'
field = winds_ctrl.V[:,:,:].mean(dim='time')

fig = map_robinson(xr_DataArray=field, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

# Trends

In [ ]:
winds_ctrl.U

In [ ]:
# # from xr_regression import atm_field_regression
# U_trends_ctrl = xr_linear_trends_2D(winds_ctrl.U, ('lat', 'lon'))
# U_trends_ctrl.to_netcdf(path=f'{path_results}/winds/U_trend_ctrl.nc' , mode='w')

# V_trends_ctrl = xr_linear_trends_2D(winds_ctrl.V, ('lat', 'lon'))
# V_trends_ctrl.to_netcdf(path=f'{path_results}/winds/V_trend_ctrl.nc' , mode='w')

# U_trends_rcp  = xr_linear_trends_2D(winds_rcp.U, ('lat', 'lon'))
# U_trends_rcp.to_netcdf(path=f'{path_results}/winds/U_trend_rcp.nc' , mode='w')

# V_trends_rcp  = xr_linear_trends_2D(winds_rcp.V, ('lat', 'lon'))
# V_trends_rcp.to_netcdf(path=f'{path_results}/winds/V_trend_rcp.nc' , mode='w')

In [ ]:
winds_ctrl.U

In [ ]:
U_trends_ctrl = xr.open_dataarray(f'{path_results}/winds/U_trend_ctrl.nc')
U_trends_rcp  = xr.open_dataarray(f'{path_results}/winds/U_trend_rcp.nc')

In [ ]:
for i, run in enumerate(['ctrl', 'rcp']):
    label = 'zonal wind velocity trend [m/s/century]'
    minv, maxv = -2.5, 2.5
    cmap = 'RdBu_r'
    filename = f'{path_results}/winds/U_trend_map_{run}'
    field = [U_trends_ctrl, U_trends_rcp][i]*100*365
    fig = map_robinson(xr_DataArray=field, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axvline(0, c='k', lw=.5)
plt.plot(100*365*xr_linear_trend(winds_ctrl.U[:,:,:].mean(dim='lon')), winds_ctrl.lat, lw=3, label='CTRL')
plt.plot(100*365*xr_linear_trend(winds_rcp.U[:,:,:].mean(dim='lon')) , winds_rcp.lat , lw=3, label='RCP')
plt.legend(fontsize=16)
plt.yticks(np.arange(-90,91,30))
plt.xlabel('mean zonal wind trend [m/s/century]', fontsize=16)
plt.ylabel('latitude [deg N]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/winds/U_trends_zonal_avg')